Проект: вариант 1

 Представьте, что вы работаете в компании, которая разрабатывает мобильные игры. К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения:

1. В первую очередь, его интересует показатель retention. Напишите функцию для его подсчета.
2. Помимо этого, в компании провели A/B тестирование наборов акционных предложений. На основе имеющихся данных определите, какой набор можно считать лучшим и на основе каких метрик стоит принять правильное решение.
3. Предложите метрики для оценки результатов последнего прошедшего тематического события в игре.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
from datetime import datetime
from datetime import timedelta
import time

import requests
from urllib.parse import urlencode
import json
from scipy.stats import levene
import pingouin as pg
from tqdm.auto import tqdm
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

## Задание 1
### Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока).

In [2]:
# считаем данные
# reg - данные о времени регистрации
reg = pd.read_csv('../shared/problem1-reg_data.csv', sep = ';')
# auth - данные о времени захода пользователей в игру
auth = pd.read_csv('../shared/problem1-auth_data.csv', sep = ';')

**Для создания функции, рассчитывающей Retention нам потребуется id пользователей, дата регистрации, захода в игру, когортный период (у игр обычно рассчитывается на 1,7, 28 день, поэтому в качестве когортного периода возьмем месяц)**

Для начала проведем EDA-анализ

In [3]:
reg.head()

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5


In [4]:
auth.head()

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2


In [5]:
reg.shape

(1000000, 2)

In [6]:
auth.shape

(9601013, 2)

In [7]:
reg.reg_ts.nunique()

1000000

In [8]:
# проверим типы значений и наличие пропущенных значений
reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   reg_ts  1000000 non-null  int64
 1   uid     1000000 non-null  int64
dtypes: int64(2)
memory usage: 15.3 MB


In [9]:
auth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
dtypes: int64(2)
memory usage: 146.5 MB


In [10]:
# переведем столбцы timestamp формата в формат даты
reg['reg_ts'] = reg['reg_ts'].apply(datetime.fromtimestamp)
auth['auth_ts'] = auth['auth_ts'].apply(datetime.fromtimestamp)

In [11]:
reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   reg_ts  1000000 non-null  datetime64[ns]
 1   uid     1000000 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 15.3 MB


In [12]:
reg.head()

,reg_ts,uid
0,1998-11-18 12:43:43,1
1,1999-07-23 02:38:09,2
2,2000-01-14 01:27:27,3
3,2000-05-28 18:19:01,4
4,2000-09-16 15:21:53,5


In [13]:
# посмотрим на ежегодное кол-во регистраций
reg.groupby(reg.reg_ts.dt.year).reg_ts.count()

reg_ts
1998         1
1999         1
2000         4
2001         6
2002        10
2003        20
2004        35
2005        65
2006       119
2007       216
2008       394
2009       718
2010      1308
2011      2383
2012      4360
2013      7931
2014     14452
2015     26339
2016     48178
2017     87626
2018    159697
2019    291041
2020    355096
Name: reg_ts, dtype: int64

In [14]:
reg.query('reg_ts >= "2019-01-01" and reg_ts < "2019-12-31"')
# посмотрели до какого дня 2019 года данные

,reg_ts,uid
353863,2019-01-01 00:02:25,392991
353864,2019-01-01 00:04:53,392992
353865,2019-01-01 00:07:22,392993
353866,2019-01-01 00:09:50,392994
353867,2019-01-01 00:12:19,392995
...,...,...
643839,2019-12-30 23:53:24,715004
643840,2019-12-30 23:54:46,715005
643841,2019-12-30 23:56:07,715006
643842,2019-12-30 23:57:29,715007


In [ ]:
# самое большое количество регистраций в 2019 году, поэтому в качестве сэмпла для расчета ретеншн возьмем данные с 2019 года
reg2019 = reg.query('reg_ts >= "2019-01-01" and reg_ts < "2019-12-30"')
auth2019 = auth.query('auth_ts >= "2019-01-01" and auth_ts < "2019-12-30"')

In [ ]:
def retention_rate(reg, auth, uid, date_reg, date_auth, period):
    
    df = reg.merge(auth, on=uid)  # Соединим данные
    
    df['cohort'] = df[date_reg].dt.to_period(period)  # Когортами будут являться месяца дат регистрации
    
    df['retention_day'] = (df[date_auth] - df[date_reg]).dt.days+1  # ретеншн с учетом дня регистрации
    
    # ежедневное количество уникальных входов после регистрации по каждой когорте
    count_cohort = df.pivot_table(index='cohort', columns='retention_day', values='uid', aggfunc='nunique')
    
    # доли входов каждый день после регистрации
    retention = count_cohort.divide(count_cohort.iloc[:,0], axis=0)
    
    #строим график
    plt.figure(figsize=(18,14))
    plt.title('Retention')
    ax = sns.heatmap(data=retention.iloc[:,0:31], 
                     annot=True, fmt='.0%', vmin=0.0, vmax=1,
                     cmap=['#B0E0E6', '#87CEFA', '#1E90FF', '#4169E1', '#0000FF'])
    ax.set_yticklabels(retention.index)
    
    
    return count_cohort.iloc[:,0:31] 

In [ ]:
# запускаем функцию
retention = retention_rate(
                                          reg = reg2019, 
                                          auth = auth2019, 
                                          uid = 'uid',
                                          date_reg = 'reg_ts',
                                          date_auth = 'auth_ts', 
                                        period = 'M')

## Задание 2
### Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.
### Какой набор предложений можно считать лучшим? Какие метрики стоит проанализировать для принятия правильного решения и как?

In [ ]:
# Для начала загрузим все датасеты
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/SOkIsD5A8xlI7Q'  # ссылка на датасет

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url) # запрос ссылки на скачивание
download_url = response.json()['href'] #'парсинг' ссылки на скачивание

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
ab = pd.read_csv(download_url, sep = ';')
ab

In [ ]:
ab.info()

In [ ]:
ab.user_id.nunique()
# количество пользователей совпадает с кол-вом строк дф ab, следовательно все пользователи уникальные

In [ ]:
# какая из двух групп тестовая?контрольная?выясним через кол-во платящих (которое дано в условии задачи)
ab.query("revenue>0").groupby('testgroup', as_index = False) \
          .agg({'revenue': 'sum', 'user_id':'count'}) \
          .rename(columns = {'user_id': 'number_of_users'})
# согласно условию в контрольной группе 1928 пользователей, следовательно "а" - контрольная группа, "b" - тестовая группа

***Для проведения тестов выдвинем две гипотезы:***

***1.Нулевая, Н0 - после проведения эксперимента значение выручки меньше у контрольной группы, т.е. набор акционных предложений в контрольной группе хуже, чем у тестовой группы;***

***2.Альтернативная, Н1 - после проведения эксперимента значение выручки у тестовой группы выше, т.е. набор акционных предложений в тестовой группе лучше, чем у контрольной группы***

In [ ]:
# посмотрим на распределение данных в контрольной группе

ab.query('testgroup == "a"').revenue.hist()
ab.query('testgroup == "a"').revenue.describe()
# тк очень много пользователей не оплатили, распределение неравномерное. Также получится и в тестовой группе

In [ ]:
# посмотрим на распределение данных в тестовой группе
ab.query('testgroup == "b"').revenue.hist()
ab.query('testgroup == "b"').revenue.describe()

In [ ]:
# посмотрим на распределение данных в контрольной группе платящих игроков
(ab.query("(testgroup == 'a') & revenue > 0")).revenue.hist()
ab.query("(testgroup == 'a') & revenue > 0").revenue.describe()
# имеются выбросы

In [ ]:
# посмотрим на распределение данных в контрольной группе платящих игроков
(ab.query("(testgroup == 'b') & revenue > 0")).revenue.hist()
ab.query("(testgroup == 'b') & revenue > 0").revenue.describe()

In [ ]:
#посмотрим на кол-во выбросов
ab.query("(testgroup == 'a') & revenue > 35000")

1.Проверим можно ли провести t-тест.Для этого должны соблюдаться следующие требования:

    -нормальность распределений
    -гомогенность дисперсий

In [ ]:
# проверка на нормальность через пингвина
pg.normality(ab, dv='revenue', group='testgroup', method="normaltest")
# normal = False => распределение ненормальное

In [ ]:
# проверка на нормальность у платящих игроков
pg.normality(ab.query("revenue>0"), dv='revenue', group='testgroup', method="normaltest")
# normal = False => распределение ненормальное

In [ ]:
# проведем тест Левена
levene(ab.query("testgroup == 'a'").revenue, ab.query("testgroup == 'b'").revenue)
# p>0.05 => отвергаем гипотезу о равенстве дисперсий

***Т.к. распределения не нормальны и дисперсии не равны (а это два главных требования для применения t-теста), то проводить t-тест не целесообразно***

**3.Для проведения бутстрапа нет жестких требований и можно анализировать любые статистики.
У нас имеются выбросы, в таких случаях медиана дает более реалистичную оценку центрального значения, чем среднее.
Проверим нулевую гипотезу о равенстве медиан (ARPPU)**

In [ ]:
def get_bootstrap(
    data_column_1, # числовые значения первой выборки
    data_column_2, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.mean, # интересующая нас статистика
    bootstrap_conf_level = 0.95 # уровень значимости
):
    boot_len = max([len(data_column_1), len(data_column_2)])
    boot_data = []
    for i in tqdm(range(boot_it)): # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            boot_len, 
            replace = True # параметр возвращения
        ).values
        
        samples_2 = data_column_2.sample(
            boot_len, # чтобы сохранить дисперсию, берем такой же размер выборки
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1-samples_2)) 
    pd_boot_data = pd.DataFrame(boot_data)
        
    left_quant = (1 - bootstrap_conf_level)/2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    quants = pd_boot_data.quantile([left_quant, right_quant])
        
    p_1 = norm.cdf(
        x = 0, 
        loc = np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2
        
    # Визуализация
    _, _, bars = plt.hist(pd_boot_data[0], bins = 50)
    for bar in bars:
        if abs(bar.get_x()) <= quants.iloc[0][0] or abs(bar.get_x()) >= quants.iloc[1][0]:
            bar.set_facecolor('red')
        else: 
            bar.set_facecolor('grey')
            bar.set_edgecolor('black')
    
    plt.style.use('ggplot')
    plt.vlines(quants,ymin=0,ymax=50,linestyle='--')
    plt.xlabel('boot_data')
    plt.ylabel('frequency')
    plt.title("Histogram of boot_data")
    plt.show()
       
    return {"boot_data": boot_data, 
            "quants": quants, 
            "p_value": p_value}

In [ ]:
get_bootstrap(
    (ab.query("(testgroup == 'a') & revenue > 0")).revenue, # числовые значения первой выборки
    (ab.query("(testgroup == 'b') & revenue > 0")).revenue, # числовые значения второй выборки
    boot_it = 1000, # количество бутстрэп-подвыборок
    statistic = np.median, # интересующая нас статистика
    bootstrap_conf_level = 0.95 # уровень значимости
)

***p_value': 0.0 < 0.05,  а также Нолик не входит в доверительный интервал => медианы значимо отличаются*** (для уточнения: медиана у платящих игроков в контрольной группе = 311, а в тестовой группе = 3022) и это нам говорит о  том, что мы можем отклонить нулевую гипотезу и сделать ***вывод: набор предложений в тестовой группе лучше набора контрольной группы***, т.к. игроки совершали более дорогие покупки.
Дополнительно для полного анализа необходимо проанализировать:
- не баг ли 123 игрока со средним чеком > 35000р.;
- если нет, то данные только с акции или до нее; 
- товары одной ли категории участвовали в акции;
- возможно цель была продать товары со стоимостью >= 35000р.

## Задание 3
### В игре Plants & Gardens каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы для сада и персонажей, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?

**Воронка конверсии пользователей** анализируем как пользователи проходят через определенную последовательность действий в игре, на каких шагах и какая часть из них отваливается (с изменением сложности уровня и тп).

**Retention и Rolling Retention**. Необходимо сегментировать на когорты и сравнивать их retention. 

**DAU** - количество уникальных пользователей, которые зашли в приложение в течение суток.

**MAU**  — количество уникальных пользователей, которые зашли в приложение в течение месяца.
По отношению средней дневной аудитории к месячной можно понимать частоту использования продукта (**Sticky Factor** - позволяет оценить регулярность посещений и стабильность пользовательской базы).

**ASL** – среднее арифметическое длин всех сессий. сравнение средней продолжительности сессии в обычный день и тематический позволит определить как тематическое событие влияет на время, которое пользователь проводит в приложении.


### Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?
Я бы оставила те же метрики и провела A/B тест, сравнила бы ретеншн до и после усложнения механики.